In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import sklearn
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.4.0
sys.version_info(major=3, minor=7, micro=9, releaselevel='final', serial=0)
matplotlib 3.3.3
numpy 1.19.2
pandas 1.2.0
sklearn 0.24.0
tensorflow 2.4.0
tensorflow.keras 2.4.0


In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(xtrain_all, ytrain_all), (xtest, ytest) = fashion_mnist.load_data()
xvalid, xtrain = xtrain_all[:5000], xtrain_all[5000:]
yvalid, ytrain = ytrain_all[:5000], ytrain_all[5000:]

print(xvalid.shape, yvalid.shape)
print(xtrain.shape, ytrain.shape)
print(xtest.shape, ytest.shape)

(5000, 28, 28) (5000,)
(55000, 28, 28) (55000,)
(10000, 28, 28) (10000,)


In [6]:
print(np.max(xtrain), np.min(xtrain))

255 0


In [ ]:
# Normalization
# x = (x - u) / std
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# xtrain [None, 28, 28] => [None, 784]
xtrain_scaled = scaler.fit_transform(
    xtrain.astype(np.float32).reshape(-1, 1)).reshape(-1, 28 * 28)
xvalid_scaled = scaler.transform(
    xvalid.astype(np.float32).reshape(-1, 1)).reshape(-1, 28 * 28)
xtest_scaled = scaler.transform(
    xtest.astype(np.float32).reshape(-1, 1)).reshape(-1, 28 * 28)

In [7]:
print(np.max(xtrain_scaled), np.min(xtrain_scaled))

2.0231433 -0.8105136


## Tensorflow 1.0

In [ ]:
hidden_units = [100, 100]  # 2 layers of 100 hidden units each
class_num = 10

x = tf.placeholder(tf.float32, [None, 28 * 28])  # None 代表 batch-size, 可以是任意值
y = tf.placeholder(tf.inf64, [None])

input_for_next_layer = x
for hidden_unit in hidden_units:
    input_for_nect_layer = tf.layers.dense(input_for_next_layer, hidden_unit, activation=tf.nn.relu)

# last hidden layer * W == (logits) -> softmax -> prob
# 1. logit -> softmax -> prob
# 2. labels -> one_hot
# 3. calculate cross entropy
logits = tf.layers.dense(input_for_next_layer, class_num)
loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=logits)
prediction = tf.argmax(logits, 1)
correct_prediction = tf.equal(prediction, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [ ]:
print(x)
print(logits)

In [ ]:
# 语言接口 称之为 客户端
# session
init = tf.global_variables_initializer()
batch_size = 20
epochs = 10
train_steps_per_epoch = xtrain.shape[0] // batch_size
valid_steps = x_valid.shape[0] // batch_size

def eval_with_sess(sess, x, y, accuracy, images, labels, batch_size):
    eval_steps = images.shape[0] // batch_size
    eval_accuracies = []
    for step in range(eval_steps):
        batch_data = images[step*batch_size: (step+1)*batch_size]
        batch_label = labels[step*batch_size: (step+1)*batch_size]
        # 这里不需要 train_op 因为不需要训练
        accuracy_val = sess.run(accuracy, feed_dict={x: batch_data, y: batch_label})
        eval_accuracies.append(accuracy_val)
    return np.mean(eval_accuracies)

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(epochs):
        for step in range(train_steps_per_epoch):
            batch_data = xtrain_scaled[step*batch_size: (step+1)*batch_size]
            batch_label = ytrain[step*batch_size: (step+1)*batch_size]
            loss_val, accuracy_val, _ = sess.run([loss, accuracy, train_op], feed_dict={
                x: batch_data, y: batch_label})
            print('\r[Train] epoch: %d, step: %d, loss: %3.5f , accuracy: %2.2f' % 
                 (epoch, step, loss_val, accuracy_val), end="")
        valid_accuracy = eval_with_sess(sess, x, y, xvalid_scaled, yvalid, batch_size)
        print("\t[Valid] acc: %2.2f" % (valid_accuracy))